<a href="https://colab.research.google.com/github/kaledai069/Crossword-Generator/blob/master/Crossword_Puzzle_Generator_with_Back_Tracking_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
class Variable():

    ACROSS = "across"
    DOWN = "down"

    def __init__(self, i, j, direction, length):
        """Create a new variable with starting point, direction, and length."""
        self.i = i
        self.j = j
        self.direction = direction
        self.length = length
        self.cells = []
        for k in range(self.length):
            self.cells.append(
                (self.i + (k if self.direction == Variable.DOWN else 0),
                 self.j + (k if self.direction == Variable.ACROSS else 0))
            )

    def __hash__(self):
        return hash((self.i, self.j, self.direction, self.length))

    def __eq__(self, other):
        return (
            (self.i == other.i) and
            (self.j == other.j) and
            (self.direction == other.direction) and
            (self.length == other.length)
        )

    def __str__(self):
        return f"({self.i}, {self.j}) {self.direction} : {self.length}"

    def __repr__(self):
        direction = repr(self.direction)
        return f"Variable({self.i}, {self.j}, {direction}, {self.length})"

class Crossword():
    def __init__(self, grid, words_file):
        self.structure = []
        self.height = len(grid) # the number of rows in the grid
        self.width = len(grid[0]) # the number of columns in the grid
        for i in range(len(grid)):
            row = []
            for j in range(len(grid[0])):
                if grid[i][j] == '':
                  row.append(False)
                else:
                  row.append(True)
            self.structure.append(row)

        # Save vocabulary list
        with open(words_file) as f:
            self.words = set(f.read().upper().splitlines())

        # Determine variable set
        self.variables = set()

        for i in range(self.height):
            for j in range(self.width):

                # Vertical words
                starts_word = (
                    self.structure[i][j]
                    and (i == 0 or not self.structure[i - 1][j])
                )
                if starts_word:
                    length = 1
                    for k in range(i + 1, self.height):
                        if self.structure[k][j]:
                            length += 1
                        else:
                            break
                    if length > 1:
                        self.variables.add(Variable(
                            i=i, j=j,
                            direction=Variable.DOWN,
                            length=length
                        ))

                # Horizontal words
                starts_word = (
                    self.structure[i][j]
                    and (j == 0 or not self.structure[i][j - 1])
                )
                if starts_word:
                    length = 1
                    for k in range(j + 1, self.width):
                        if self.structure[i][k]:
                            length += 1
                        else:
                            break
                    if length > 1:
                        self.variables.add(Variable(
                            i=i, j=j,
                            direction=Variable.ACROSS,
                            length=length
                        ))

        # Compute overlaps for each word
        # For any pair of variables v1, v2, their overlap is either:
        #    None, if the two variables do not overlap; or
        #    (i, j), where v1's ith character overlaps v2's jth character
        self.overlaps = dict()
        for v1 in self.variables:
            for v2 in self.variables:
                if v1 == v2:
                    continue
                cells1 = v1.cells
                cells2 = v2.cells
                intersection = set(cells1).intersection(cells2)
                if not intersection:
                    self.overlaps[v1, v2] = None
                else:
                    intersection = intersection.pop()
                    self.overlaps[v1, v2] = (
                        cells1.index(intersection),
                        cells2.index(intersection)
                    )

    def neighbors(self, var):
        """Given a variable, return set of overlapping variables."""
        return set(
            v for v in self.variables
            if v != var and self.overlaps[v, var]
        )

In [83]:
import copy
from collections import deque
from queue import PriorityQueue
import random

class CrosswordCreator():

    def __init__(self, crossword):
        """
        Create new CSP crossword generate.
        """
        self.crossword = crossword
        self.domains = {
            var: get_required_length_answers(var.length)
            for var in self.crossword.variables
        }

    def get_required_length_answers(self, ans_length):
        output = []
        for word in self.crossword.words:
            if len(word) == ans_length:
                output.append(word)

        random.shuffle(output)
        return set(output)

    def letter_grid(self, assignment):
        """
        Return 2D array representing a given assignment.
        """
        letters = [
            [None for _ in range(self.crossword.width)]
            for _ in range(self.crossword.height)
        ]
        for variable, word in assignment.items():
            direction = variable.direction
            for k in range(len(word)):
                i = variable.i + (k if direction == Variable.DOWN else 0)
                j = variable.j + (k if direction == Variable.ACROSS else 0)
                letters[i][j] = word[k]
        return letters

    def print(self, assignment):
        """
        Print crossword assignment to the terminal.
        """
        letters = self.letter_grid(assignment)
        for i in range(self.crossword.height):
            for j in range(self.crossword.width):
                if self.crossword.structure[i][j]:
                    print(letters[i][j] or " ", end="")
                else:
                    print("█", end="")
            print()

    def save(self, assignment, filename):
        """
        Save crossword assignment to an image file.
        """
        from PIL import Image, ImageDraw, ImageFont
        cell_size = 100
        cell_border = 2
        interior_size = cell_size - 2 * cell_border
        letters = self.letter_grid(assignment)

        # Create a blank canvas
        img = Image.new(
            "RGBA",
            (self.crossword.width * cell_size,
             self.crossword.height * cell_size),
            "black"
        )
        font = ImageFont.truetype("Roboto-Regular.ttf", 80)
        draw = ImageDraw.Draw(img)

        for i in range(self.crossword.height):
            for j in range(self.crossword.width):

                rect = [
                    (j * cell_size + cell_border,
                     i * cell_size + cell_border),
                    ((j + 1) * cell_size - cell_border,
                     (i + 1) * cell_size - cell_border)
                ]
                if self.crossword.structure[i][j]:
                    draw.rectangle(rect, fill="white")
                    if letters[i][j]:
                        _, _, w, h = draw.textbbox((0, 0), letters[i][j], font=font)
                        draw.text(
                            (rect[0][0] + ((interior_size - w) / 2),
                             rect[0][1] + ((interior_size - h) / 2) - 10),
                            letters[i][j], fill="black", font=font
                        )

        img.save(filename)

    def solve(self):
        """
        Enforce node and arc consistency, and then solve the CSP.
        """
        self.enforce_node_consistency()
        self.ac3()
        return self.backtrack(dict())

    def enforce_node_consistency(self):
        """
        Update `self.domains` such that each variable is node-consistent.
        (Remove any values that are inconsistent with a variable's unary
         constraints; in this case, the length of the word.)
        """
        for variable in self.crossword.variables:
            valid_words = set()
            for word in self.domains[variable]:
                if len(word) == variable.length:
                    valid_words.add(word)
            self.domains[variable] = valid_words


    # def revise(self, x, y):
    #     """
    #     Make variable `x` arc consistent with variable `y`.
    #     To do so, remove values from `self.domains[x]` for which there is no
    #     possible corresponding value for `y` in `self.domains[y]`.

    #     Return True if a revision was made to the domain of `x`; return
    #     False if no revision was made.
    #     """
    #     revised = False
    #     new_domain = set()
    #     overlap = self.crossword.overlaps[x, y]
    #     if overlap:
    #         for word in self.domains[x]:
    #             if word[overlap[0]] in [wordy[overlap[1]] for wordy in self.domains[y]]:
    #                 new_domain.add(word)
    #             else:
    #                 revised = True
    #         self.domains[x] = new_domain
    #     return revised

    # phind AI
    def revise(self, x, y):
        revised = False
        overlap = self.crossword.overlaps[x, y]
        y_chars = {word[overlap[1]] for word in self.domains[y]}  # Precompute the y's second character set

        self.domains[x] = {word for word in self.domains[x] if word[overlap[0]] in y_chars}
        if len(self.domains[x]) < len(self.domains[y]):
            revised = True
        return revised

    # def revise(self, x, y):
    #     """
    #     Make variable `x` arc consistent with variable `y`.
    #     To do so, remove values from `self.domains[x]` for which there is no
    #     possible corresponding value for `y` in `self.domains[y]`.

    #     Return True if a revision was made to the domain of `x`; return
    #     False if no revision was made.
    #     """
    #     revised = False
    #     new_domain = set()
    #     overlap = self.crossword.overlaps[x, y]
    #     if overlap:
    #         print(f"Checking overlap: {overlap}")
    #         for word in self.domains[x]:
    #             if any(word[overlap[0]] == wordy[overlap[1]] for wordy in self.domains[y]):
    #                 new_domain.add(word)
    #             else:
    #                 revised = True
    #         self.domains[x] = new_domain
    #         print(f"Revised domain of {x}: {list(self.domains[x])[:5]}")
    #     return revised

    # def ac3(self, arcs=None):
    #     """
    #     Update `self.domains` such that each variable is arc consistent.
    #     If `arcs` is None, begin with initial list of all arcs in the problem.
    #     Otherwise, use `arcs` as the initial list of arcs to make consistent.

    #     Return True if arc consistency is enforced and no domains are empty;
    #     return False if one or more domains end up empty.
    #     """
    #     if arcs == None:
    #         arcs = [(v1, v2) for v1 in self.crossword.variables for v2 in self.crossword.variables if v1 != v2]
    #     while arcs:
    #         x, y = arcs.pop(0) # dequeue
    #         if self.revise(x, y):
    #             if not self.domains[x]:
    #                 return False
    #             arcs.extend(set([(v, x) for v in self.crossword.neighbors(x)]) - {(y, x)})
    #     return True

    def ac3(self, arcs=None):
        if arcs is None:
            arcs = deque([(v1, v2) for v1 in self.crossword.variables for v2 in self.crossword.neighbors(v1)])
        else:
            arcs = deque(arcs)

        while arcs:
            x, y = arcs.popleft()  # Efficient pop from the left
            if self.revise(x, y):
                if not self.domains[x]:
                    return False
                for z in self.crossword.neighbors(x) - {y}:
                    arcs.append((z, x))
        return True

    def assignment_complete(self, assignment):
        """
        Return True if `assignment` is complete (i.e., assigns a value to each
        crossword variable); return False otherwise.
        """
        complete = True
        vars_in_assignment = set(var for var in assignment)
        # Checking if all vars in the crossword has been assigned
        if vars_in_assignment != self.crossword.variables:
            complete = False
        for var in assignment:
            # making sure no var is empty
            assert isinstance(assignment[var], str)
            if not assignment[var]:
                complete = False
        return complete

    # phind AI
    # def assignment_complete(self, assignment):
    #     return len(assignment) == len(self.crossword.variables)

    # def consistent(self, assignment):
    #     """
    #     Return True if `assignment` is consistent (i.e., words fit in crossword
    #     puzzle without conflicting characters); return False otherwise.
    #     """
    #     values = [] # Must in be a list.
    #     consistent = True
    #     for var in assignment:
    #         assert isinstance(assignment[var], str)
    #         if assignment[var] in values:
    #             consistent = False
    #             break
    #         else:
    #             values.append(assignment[var])

    #         if var.length != len(assignment[var]):
    #             consistent = False
    #             break
    #         for neighbor in self.crossword.neighbors(var):
    #             if neighbor in assignment:
    #                 overlap = self.crossword.overlaps[var, neighbor]
    #                 if assignment[var][overlap[0]] != assignment[neighbor][overlap[1]]:
    #                     consistent = False
    #                     break
    #     return consistent

    # phind AI
    def consistent(self, assignment):
        values = set()
        for var, word in assignment.items():
            if word in values or len(word) != var.length:
                return False
            values.add(word)
            for neighbor in self.crossword.neighbors(var):
                overlap = self.crossword.overlaps[var, neighbor]
                if neighbor in assignment and assignment[var][overlap[0]] != assignment[neighbor][overlap[1]]:
                    return False
        return True

    def order_domain_values(self, var, assignment):
        """
        Return a list of values in the domain of `var`, in order by
        the number of values they rule out for neighboring variables.
        The first value in the list, for example, should be the one
        that rules out the fewest values among the neighbors of `var`.
        """
        values_penalty = {value: 0 for value in self.domains[var]}
        for neighbor in self.crossword.neighbors(var):
            if neighbor not in assignment:
                overlap = self.crossword.overlaps[var, neighbor]
                for value in self.domains[var]:
                    for value2 in self.domains[neighbor]:
                        if value[overlap[0]] != value2[overlap[1]]:
                            values_penalty[value] += 1

        return sorted([value in self.domains[var]], key= lambda item: values_penalty[item])


    def select_unassigned_variable(self, assignment):
        """
        Return an unassigned variable not already part of `assignment`.
        Choose the variable with the minimum number of remaining values
        in its domain. If there is a tie, choose the variable with the highest
        degree. If there is a tie, any of the tied variables are acceptable
        return values.
        """
        var_penalty = {}
        for var in self.crossword.variables:
            if var not in assignment:
                var_penalty[var] = len(self.domains[var])
        vars = sorted(var_penalty, key= lambda v: var_penalty[v])
        # if the two first variables have the same domain size
        if len(vars) > 1 and var_penalty[vars[0]] == var_penalty[vars[1]]:
            # Check number of neighbors and return highest degree
            if len(self.crossword.neighbors(vars[0])) < len(self.crossword.neighbors(vars[1])):
                return vars[1]
        return vars[0]


    # def backtrack(self, assignment):
    #     """
    #     Using Backtracking Search, take as input a partial assignment for the
    #     crossword and return a complete assignment if possible to do so.

    #     `assignment` is a mapping from variables (keys) to words (values).

    #     If no assignment is possible, return None.
    #     """
    #     if self.assignment_complete(assignment): return assignment # base case

    #     var = self.select_unassigned_variable(assignment)

    #     for value in self.domains[var]:
    #         new_assignment = copy.deepcopy(assignment)
    #         new_assignment[var] = value
    #         if self.consistent(new_assignment):
    #             result = self.backtrack(new_assignment)
    #             if result != None: return result
    #     return None


    # Modify the backtrack method
    def backtrack(self, assignment):
        """
        Using Backtracking Search, take as input a partial assignment for the
        crossword and return a complete assignment if possible to do so.

        `assignment` is a mapping from variables (keys) to words (values).

        If no assignment is possible, return None.
        """
        if self.assignment_complete(assignment):
            return assignment  # base case

        var = self.select_unassigned_variable(assignment)

        for value in self.domains[var]:
            new_assignment = assignment.copy()  # or dict(assignment)
            new_assignment[var] = value
            if self.consistent(new_assignment):
                result = self.backtrack(new_assignment)
                if result is not None:
                    return result
        return None


In [ ]:
word_list_path = '/content/all_answers.txt'

# # 5x5 grid
grid = [['', '', 'A', 'A', 'A'],
        ['', 'A', 'A', 'A', 'A'],
        ['A', 'A', 'A', 'A', 'A'],
        ['A', 'A', 'A', 'A', ''],
        ['A', 'A', 'A', '', '']]

# 4x4 grid
# grid = [['', 'A', 'A', 'A'],
#         ['A', 'A', 'A', 'A'],
#         ['A', 'A', 'A', 'A'],
#         ['A', 'A', 'A', ''],]


crossword = Crossword(grid, word_list_path)
creator = CrosswordCreator(crossword)
assignment = creator.solve()

In [85]:
with open("/content/all_answers.txt") as f:
    words = list(set(f.read().upper().splitlines()))

for key, answer in assignment.items():
    if answer in words:
        print(answer, 'FOUND')

RGT FOUND
MOC FOUND
CIS FOUND
TSP FOUND
MNTS FOUND
GTOS FOUND
OLOP FOUND
RNLI FOUND


In [33]:
with open("/content/all_answers.txt") as f:
    words = set(f.read().upper().splitlines())

def get_required_length_answers(ans_length):
    output = set()
    for word in words:
        if len(word) == ans_length:
            output.add(word)
    return output

In [34]:
domains = {
    var: get_required_length_answers(var.length) for var in crossword.variables
}

In [36]:
for key, value in domains.items():
    print(len(value))

17222
5997
17222
17222
17222
5997
5997
5997


In [22]:
from pprint import pprint
pprint(crossword.structure)
list(crossword.variables)[0].cells


[[False, True, True, True],
 [True, True, True, True],
 [True, True, True, True],
 [True, True, True, False]]


[(0, 1), (1, 1), (2, 1), (3, 1)]